# NG Data Portal API Wrapper Development

<br>

This notebook documents the development of an API wrapper for the National Grid Data Portal. 

It starts by looking at a single data stream, containing the percentage generation of each fuel, moving from basic requests to more developed and generalisable queries. During this process potential areas for user error are identified and warnings put in place to help guide them in their usage. Finally the groups of functions are combined into a single wrapper class within the NGDataPortal script.

The next stage is to identify the different data streams and create a mapping to their resource ids.

<br>

### Imports

In [ ]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import warnings
from itertools import islice

from IPython.core.display import display, HTML

<br>

### Helper Functions

In [ ]:
def take(iterable, n=5):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

<br>

### Retrieving Package Information

We can retrieve information relating to specific data streams through their respective datapackage.jsons

In [ ]:
form_package_url = lambda group, stream: f'https://national-grid.ckan.io/{group}/{stream}/datapackage.json'

group = 'generation'
stream = 'generation-mix-national'

package_url = form_package_url(group, stream)
r_package = requests.get(package_url)

r_package

<br>

These include general information about the data stream

In [ ]:
package_desc = r_package.json()['resources'][0]['descriptionHtml']

display(HTML(package_desc))

<br>

As well as important attributes such as the resource id, with this we can now access the API data feeds

In [ ]:
resource_id = r_package.json()['resources'][0]['id']

resource_id

<br>

### Creating the API Wrapper

We start the API wrapper development by creating a function which constructs the url for the relevant endpoint and then makes a request to it.

In [ ]:
def NG_request(resource_id, params={}):    
    url_root = 'https://national-grid-admin.ckan.io/api/3/action/datastore_search'
    
    params.update({'resource_id':resource_id})
    
    if 'sql' in params.keys():
        url_root += '_sql'
        
    r = requests.get(url_root, params=params)
    
    return r

r = NG_request(resource_id)
df = pd.DataFrame(r.json()['result']['records'])

df.head(2)

<br>

Before we take this further we want to add some checks which confirm the call was successful and if not exposes the error message returned by the API. We create a wrapper for the error to enable its assignment as a dictionary value in the next section.

In [ ]:
def raise_(err_txt, error=ValueError): 
    raise error(err_txt)
    
def check_request_success(r_json):
    if r_json['success'] == False:
        err_msg = r_json['error']['message']
        raise_(err_msg)
        
r = NG_request('incorrect_resource_id')
r_json = r.json()

#check_request_success(r_json)

<br>

The most common 'advanced' query to the API will include some specification around the date range for the data, within CKAN we can use SQL to construct these specific requests.

In [ ]:
date_between = lambda dt_col, resource_id, start_date, end_date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" BETWEEN \'{start_date}\'::timestamp AND \'{end_date}\'::timestamp ORDER BY "{dt_col}"' 
date_less_than = lambda dt_col, resource_id, date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" < \'{date}\'::timestamp ORDER BY "{dt_col}"' 
date_greater_than = lambda dt_col, resource_id, date: f'SELECT * from "{resource_id}" WHERE "{dt_col}" > \'{date}\'::timestamp ORDER BY "{dt_col}"' 

def form_dt_rng_sql_query(dt_col, resource_id, start_date=None, end_date=None):
    start_end_date_exist = (start_date!=None, end_date!=None)
    
    func_map = {
        (False, False) : {'error' : 'A start and/or end date should be passed'},
        (True, True) : date_between(dt_col, resource_id, start_date, end_date),
        (False, True) : date_less_than(dt_col, resource_id, end_date),
        (True, False) : date_greater_than(dt_col, resource_id, start_date),
    }
    
    sql = func_map[start_end_date_exist]
    
    if not isinstance(sql, str):
        raise_(sql['error'])
    
    return sql
    
dt_col = 'dateTime_from'
start_date = '2019-12-30'
end_date = '2019-12-31'

sql = form_dt_rng_sql_query(dt_col, resource_id, start_date=start_date, end_date=end_date)

print(sql)

<br>

We're now ready to put this all together into a single function

In [ ]:
def query_API(resource_id, params={}, start_date=None, end_date=None, dt_col=None, sql=None, return_raw=False):
    ## Handling SQL queries
    if start_date or end_date:
        if sql:
            warnings.warn('The start and end date query will overwrite the provided SQL')
        
        if not dt_col:
            raise_('If a start or end date has been provided the \'dt_col\' parameter must be provided')
        
        sql = form_dt_rng_sql_query(dt_col, resource_id, start_date=start_date, end_date=end_date)
        params.update({'sql':sql})
    
    elif sql:
        params.update({'sql':sql})
    
    ## Making the request
    r = NG_request(resource_id, params=params)
    
    if return_raw == True:
        return r

    ## Checking and parsing the response
    r_json = r.json()
    check_request_success(r_json)
    
    df = pd.DataFrame(r_json['result']['records'])
        
    return df

start_date = '2019-12-20 12:00'
end_date = '2019-12-22'
dt_col = 'dateTime_from'

df = query_API(resource_id, start_date=start_date, end_date=end_date, dt_col=dt_col)

df.head(2)

<br>

Which makes it far easier to query any of the data streams

In [ ]:
RES_fcst_resource_id = 'db6c038f-98af-4570-ab60-24d71ebd0ae5'

df = query_API(RES_fcst_resource_id)

df.head()

<br>

This can then be used to create a module which contains all of the neccessary equations

In [ ]:
from NGDataPortal import Wrapper

RES_stream = 'embedded-wind-and-solar-forecasts'
wrapper = Wrapper(stream=RES_stream)

df = wrapper.query_API()

df.head()

<br>

### Collecting Metadata for Available Streams

In [ ]:
url_search_NGDataPortal = lambda q, from_: f'https://data.nationalgrideso.com/search?q={q}&from={from_}&sort=score:desc'

search_url = url_search_NGDataPortal('', 0)
r = requests.get(search_url)

r

In [ ]:
def response_2_resource_links(r):
    soup = BeautifulSoup(r.content)

    resource_links = soup.findAll('a', {'class':'text-orange hover:text-gray-800'})
    resource_links = [resource_link['href'] for resource_link in resource_links]

    return resource_links

resource_links = response_2_resource_links(r)

resource_links

In [ ]:
resource_links = []
num_page_resource_links = 1

while num_page_resource_links > 0:
    search_url = url_search_NGDataPortal('', len(resource_links))
    r = requests.get(search_url)
    
    page_resource_links = response_2_resource_links(r)
    num_page_resource_links = len(page_resource_links)
    
    resource_links += page_resource_links
    
print(f'There are a total of {len(resource_links)} data streams available through the National Grid portal')

df_resources = (pd.Series(resource_links)
                .str[1:]
                .str.split('/')
                .apply(pd.Series)
                .rename(columns={0:'group', 1:'stream'})
               )

df_resources.head()

In [ ]:
def package_url_2_metadata(package_url):
    r_package = requests.get(package_url)
    r_json = r_package.json()

    package_metadata = dict()

    package_metadata['package_desc'] = r_json['resources'][0]['description']
    package_metadata['resource_id'] = r_json['resources'][0]['id']

    return package_metadata

group = 'generation'
stream = 'generation-mix-national'

package_url = form_package_url(group, stream)
package_metadata = package_url_2_metadata(package_url)

package_metadata

In [ ]:
%%time

identify_resource_ids = False

if identify_resource_ids == True:
    for idx, (group, stream) in df_resources.iterrows():
        package_url = form_package_url(group, stream)
        package_metadata = package_url_2_metadata(package_url)

        df_resources.loc[idx, 'resource_id'] = package_metadata['resource_id']

    stream_2_id_map = df_resources.set_index('stream')['resource_id'].to_dict()

    print(dict(take(stream_2_id_map.items())))

In [ ]:
if identify_resource_ids == True:
    with open('stream_to_resource_id.json', 'w') as fp:
        json.dump(stream_2_id_map, fp)

In [ ]:
stream = 'current-balancing-services-use-of-system-bsuos-data'
wrapper = Wrapper(stream=stream)

start_date = '2019-12-20'
end_date = '2019-12-22'
dt_col = 'Settlement Day'

df = wrapper.query_API(start_date=start_date, end_date=end_date, dt_col=dt_col)

df.head(2)

In [ ]:
stream = 'weekly-wind-availability'
wrapper = Wrapper(stream)

df = wrapper.query_API()

df.head()

In [ ]:
stream = 'embedded-wind-and-solar-forecasts'
wrapper = Wrapper(stream)

df = wrapper.query_API()

df.head()